In [1]:
"""State management for the index graph."""
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Annotated, Optional

from langchain_core.documents import Document

from dataclasses import dataclass, field
from typing import Optional, TypedDict, List, Dict, Union, Annotated
from langchain.schema import AgentAction, AgentFinish
import operator

@dataclass
class AgentState:
    resume: str
    job_description: str
    job_title: str
    company_name: str
    history: List[Dict[str, str]] = field(default_factory=list)
    generated_question: Optional[Union[AgentAction, AgentFinish, Dict[str, str]]] = None
    human_answer: Optional[str] = None
    answer_evaluation: Optional[Union[AgentAction, AgentFinish, Dict[str, str]]] = None
    candidate_evaluation: Optional[Union[AgentAction, AgentFinish, Dict[str, str]]] = None


In [2]:
"""This "graph" simply exposes an endpoint for a user to upload docs to be indexed."""


import json
import os
from typing import Literal, Optional, Union
from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, START, StateGraph
from langchain_core.tools import tool
from langchain_openai.chat_models import AzureChatOpenAI, ChatOpenAI
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain
from langchain.tools import HumanInputRun
from langchain_core.messages.ai import AIMessage
from langchain_core.runnables.base import Runnable

from langsmith import Client
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List, Optional
from pydantic import BaseModel, Field
from typing import Optional, List, Dict

class BaseQuestion(BaseModel):
    topic: str = Field(description="Topic of the question that has been generated")
    question: str = Field(description="Interview question for the candidate")
    answer: str = Field(description="Suggested answer or evaluation criteria for the question")
    complexity: int = Field(description="A number from 1 to 5, where 1 is least complex and 5 is most complex")

class PersonalQuestion(BaseModel):
    topic: str = Field(description="Topic of the question that has been generated")
    question: str = Field(description="The full text of the HR question")
    answer: str = Field(description="A suggested model answer or evaluation criteria for the question")
    complexity: int = Field(description="A number from 1 to 5, where 1 is least complex and 5 is most complex")

class HRQuestion(BaseModel):
    topic: str = Field(description="Topic of the question that has been generated")
    question: str = Field(description="The full text of the HR question")
    answer: str = Field(description="A suggested model answer or evaluation criteria for the question")
    complexity: int = Field(description="A number from 1 to 5, where 1 is least complex and 5 is most complex")

class CriticalQuestion(BaseModel):
    topic: str = Field(description="Topic of the question that has been generated")
    scenario: str = Field(description="A detailed description of the situation or problem")
    question: str = Field(description="The specific question or task for the candidate based on the scenario")
    key_considerations: List[str] = Field(description="A list of important factors the candidate should consider in their response")
    evaluation_criteria: str = Field(description="Key points or approaches the candidate should demonstrate in their answer")
    follow_up: Optional[str] = Field(description="An optional follow-up question to probe deeper into the candidate's thinking")
    complexity: int = Field(description="A number from 1 to 5, where 1 is least complex and 5 is most complex")

class TechnicalQuestion(BaseModel):
    topic: str = Field(description="Topic of the question that has been generated")
    question_type: Literal["Theoretical", "Coding Challenge", "System Architecture"] = Field(description="Type of technical question")
    question: str = Field(description="The full text of the technical question or coding challenge")
    instructions: str = Field(description="Clear, step-by-step instructions for the candidate")
    example_input: Optional[str] = Field(description="Sample input for the problem, if applicable")
    example_output: Optional[str] = Field(description="Expected output for the sample input, if applicable")
    answer_criteria: str = Field(description="Key points or approach the candidate should demonstrate in their answer or solution")
    complexity: int = Field(description="A number from 1 to 5, where 1 is least complex and 5 is most complex")


class EvaluateAnswers(BaseModel):
    question: str = Field(description="Question for which the evaluation is made")
    answer: str = Field(description="Answer for which the evaluation is made(human entered answer)")
    evaluation_summary: str = Field(description="Evaluation summary for the given question and answer that has been generated")
    relevance: int = Field(description="Relevance score of the answer")
    correctness: int = Field(description="Correctness score of the answer")
    leadership: int = Field(description="Leadership score assessed from the answer")
    team_work: int = Field(description="Team work score assessed from the answer")
    technical_strength: int = Field(description="Technical strength score assessed from the answer")
    communication: int = Field(description="Communication score assessed from the answer")

class CategoryEvaluation(BaseModel):
    score: int = Field(..., description="Score from 1 (Poor) to 5 (Excellent)")
    examples: List[str] = Field(..., description="Specific examples supporting the evaluation")

class CandidateEvaluation(BaseModel):
    overall_assessment: str = Field(..., description="Concise summary of the candidate's suitability for the job role")
    
    technical_proficiency: CategoryEvaluation = Field(
        ..., description="Evaluation of technical knowledge, problem-solving skills, and technical challenge handling (Weight: 25%)"
    )
    communication_skills: CategoryEvaluation = Field(
        ..., description="Evaluation of communication clarity, effectiveness, and ability to explain complex concepts (Weight: 20%)"
    )
    cultural_fit: CategoryEvaluation = Field(
        ..., description="Assessment of alignment with company culture and potential contribution to the team (Weight: 15%)"
    )
    leadership_teamwork: CategoryEvaluation = Field(
        ..., description="Evaluation of leadership potential and ability to collaborate effectively (Weight: 15%)"
    )
    adaptability_learning_agility: CategoryEvaluation = Field(
        ..., description="Assessment of adaptability, learning agility, openness to feedback, and continuous improvement (Weight: 15%)"
    )
    relevant_experience: CategoryEvaluation = Field(
        ..., description="Evaluation of alignment between past experiences and job role requirements (Weight: 10%)"
    )
    
    strengths: List[str] = Field(..., description="3-5 key strengths of the candidate with specific examples")
    areas_for_improvement: List[str] = Field(..., description="2-3 areas where the candidate could improve or need support")
    
    final_recommendation: str = Field(
        ..., description="Final recommendation: Strongly Recommend Hire, Recommend Hire, Recommend Additional Interview, or Do Not Recommend"
    )
    
    additional_comments: Optional[str] = Field(None, description="Additional insights or observations relevant to the hiring decision")
    
load_dotenv()

client = Client()

model = ChatOpenAI(temperature=0,
                            model_kwargs={"seed": 42},
                            streaming=True,
                            api_key=os.environ.get("OPENAI_API_KEY")
                            )

def personal_questions(state: AgentState) -> Dict[str, Union[PersonalQuestion, str]]:
    """Generate interview questions using an LLM."""
    try:
        document_prompt = client.pull_prompt("interviewer_personal_question")
        print(document_prompt)
        chain = (document_prompt | model.with_structured_output(PersonalQuestion))
        response = chain.invoke({
            "resume": state.resume,
            "job_description": state.job_description,
            "job_title": state.job_title,
            "company_name": state.company_name,
            "history": state.history
        })
        print(response)

        if not all(hasattr(response, field) for field in ["topic", "question"]):
            raise ValueError("Generated question is missing required fields")

        return {"generated_question": response}
    except Exception as e:
        print(f"Error in generate_questions: {e}")
        return {"generated_question": f"Could not generate questions. Details: {str(e)}"}
    
def critical_questions(state: AgentState) -> Dict[str, Union[CriticalQuestion, str]]:
    """Generate interview questions using an LLM."""
    try:
        question_count = len(state.history)

        document_prompt = client.pull_prompt("interviewer_critical_questions")
        chain = (document_prompt | model.with_structured_output(CriticalQuestion))
        response = chain.invoke({
            "resume": state.resume,
            "job_description": state.job_description,
            "job_title": state.job_title,
            "company_name": state.company_name,
            "history": state.history,
            "question_count": question_count
        })

        if not all(hasattr(response, field) for field in ["topic", "question"]):
            raise ValueError("Generated question is missing required fields")

        return {"generated_question": response}
    except Exception as e:
        print(f"Error in generate_questions: {e}")
        return {"error": f"Could not generate questions. Details: {str(e)}"}

def technical_questions(state: AgentState) -> Dict[str, Union[TechnicalQuestion, str]]:
    """Generate interview questions using an LLM."""
    try:
        question_count = len(state.history)

        document_prompt = client.pull_prompt("interviewer_technical_questions")
        chain = (document_prompt | model.with_structured_output(TechnicalQuestion))
        response = chain.invoke({
            "resume": state.resume,
            "job_description": state.job_description,
            "job_title": state.job_title,
            "company_name": state.company_name,
            "history": state.history,
            "question_count": question_count
        })

        if not all(hasattr(response, field) for field in ["topic", "question"]):
            raise ValueError("Generated question is missing required fields")

        return {"generated_question": response}
    except Exception as e:
        print(f"Error in generate_questions: {e}")
        return {"error": f"Could not generate questions. Details: {str(e)}"}
    
def hr_questions(state: AgentState) -> Dict[str, Union[HRQuestion, str]]:
    """Generate interview questions using an LLM."""
    try:
        question_count = len(state.history)

        document_prompt = client.pull_prompt("interviewer_hr_questions")
        chain = (document_prompt | model.with_structured_output(HRQuestion))
        response = chain.invoke({
            "resume": state.resume,
            "job_description": state.job_description,
            "job_title": state.job_title,
            "company_name": state.company_name,
            "history": state.history,
            "question_count": question_count
        })

        if not all(hasattr(response, field) for field in ["topic", "question"]):
            raise ValueError("Generated question is missing required fields")

        return {"generated_question": response}
    except Exception as e:
        print(f"Error in generate_questions: {e}")
        return {"error": f"Could not generate questions. Details: {str(e)}"}

def human_input(state: AgentState) -> AgentState:
    """Accepts a human response and updates the state."""
    max_retries = 3
    retries = 0

    while retries < max_retries:
        try:
            response = input()
            
            if not response.strip():
                print("Input cannot be empty. Please provide a valid response.")
                retries += 1
                continue
            
            state.human_answer = response
            return state
        except EOFError:
            print("EOFError: Input stream is closed or unavailable.")
            retry = input("Do you want to retry? (y/n): ").lower()
            if retry != 'y':
                raise ValueError("User chose to exit after encountering an EOFError.")
            retries += 1

    raise ValueError("Maximum retries reached. Unable to get valid input.")

def reset_entries(state: AgentState) -> AgentState:
    """Resets specific entries in the state."""
    state.generated_question = None
    state.answer_evaluation = None
    state.human_answer = None
    return state

def evaluate_answers(state: AgentState) -> Dict[str, Union[EvaluateAnswers, str]]:
    """Generate evaluation for the human answer."""
    try:
        document_prompt = client.pull_prompt("interviewer_answer_evaluator_prompt")
        chain = (document_prompt | model.with_structured_output(EvaluateAnswers))
        evaluation_response = chain.invoke({
            "resume": state.resume,
            "job_description": state.job_description,
            "job_title": state.job_title,
            "company_name": state.company_name,
            "question": state.generated_question.question if isinstance(state.generated_question) else str(state.generated_question),
            "answer": state.human_answer,
            "history": state.history
        })
        
        state.history.append(evaluation_response.dict())
        return {"answer_evaluation": evaluation_response}
    except Exception as e:
        print(f"Error in evaluate_answers: {e}")
        return {"answer_evaluation": f"Could not process question and answer. Details: {str(e)}"}

def count_questions(state: AgentState) -> str:
    """Determines whether to finish based on the number of questions asked."""
    if len(state.history) >= 20:
        return "evaluate_candidate"
    else:
        return "generate_questions"
    
def evaluate_candidate(state: AgentState)-> Dict[str, Union[CandidateEvaluation, str]]:
    """Generate candidate evaluation using an LLM."""
    try:
        document_prompt = client.pull_prompt("candidate_evaluation_prompt")
        chain = (document_prompt | model.with_structured_output(CandidateEvaluation))
        response = chain.invoke({ "job_description": state.job_description,
                           "job_role": state.job_title,
                           "company_name": state.company_name,
                           "history": state.history})
        return {"candidate_evaluation": response}
    except Exception as e:
        print("Error:", e)
        return {"candidate_evaluation": f"Could not generate candidate_evaluation. Details: {str(e)}"}
    
def questions_topic_decider(state: AgentState) -> str:
    """Determines which topic to choose based on the number of questions asked."""
    if len(state.history) <= 7:
        return "personal_questions"
    elif len(state.history) <= 20:
        return "technical_questions"
    elif len(state.history) <= 27:
        return "critical_questions"
    else:
        return "hr_questions" 
    
# Define the graph
builder = StateGraph(AgentState)
builder.add_node("technical_questions", technical_questions)
builder.add_node("hr_questions", hr_questions)
builder.add_node("critical_questions", critical_questions)
builder.add_node("personal_questions", personal_questions)
builder.add_node("human_input", human_input)
builder.add_node("evaluate_answers", evaluate_answers)
builder.add_node("reset_entries", reset_entries)
builder.add_node("evaluate_candidate", evaluate_candidate)


builder.add_edge(START, "personal_questions")
builder.add_edge("personal_questions", "human_input")
builder.add_edge("technical_questions", "human_input")
builder.add_edge("critical_questions", "human_input")
builder.add_edge("hr_questions", "human_input")
builder.add_edge("human_input", "evaluate_answers")

builder.add_conditional_edges(
    "reset_entries",
    questions_topic_decider,
    {
        "personal_questions": "personal_questions",
        "technical_questions": "technical_questions",
        "critical_questions": "critical_questions",
        "hr_questions": "hr_questions"
    }
)

builder.add_conditional_edges(
    "evaluate_answers", 
    count_questions,   
    {                   
        "generate_questions": "reset_entries", 
        "evaluate_candidate": "evaluate_candidate"
    }
)

builder.add_edge("evaluate_candidate", END)

# Compile into a graph object that you can invoke and deploy.
graph = builder.compile()
graph.name = "IndexGraph"



agent_state = AgentState(
    resume="John Doe's resume text here...",
    job_description="Job description for a data scientist role...",
    job_title="Data Scientist",
    company_name="Tech Corp",
    history=[{"question": "What are your strengths?", "answer": "Problem-solving and teamwork"}]
)

result = graph.invoke({
    "resume": """Name: Rahul Krishnamoorthy
Email: rahul.krish@example.com
Phone: +1-234-567-8901
LinkedIn: linkedin.com/in/rahulkrish
GitHub: github.com/rahulkrish

Professional Summary
Dynamic AI Engineer and Full-Stack Developer with 5+ years of experience designing, developing, and deploying innovative software solutions. Proficient in backend development, API integration, and building data pipelines. Skilled in integrating GPT models and deploying CI/CD pipelines using AWS services.

Technical Skills
Programming Languages: Python, JavaScript, Java
Frameworks & Tools: FastAPI, Flask, Vue.js, LangChain
Cloud Technologies: AWS (EKS, S3, SES, MSK)
Databases: MongoDB, PostgreSQL
Other Skills: Kafka, CI/CD pipelines, Generative AI Integration
Experience
AI Engineer | Oros Tech LLC (Remote)
Jan 2022 – Present

Developed over 50+ RESTful APIs using FastAPI and Flask.
Built streaming pipelines using Kafka for real-time data processing.
Integrated GPT-4 with LangChain to create intelligent question-generation systems for recruitment.
Deployed applications on AWS infrastructure using Docker and Kubernetes.
Service Engineer | 247.ai
Jun 2019 – Dec 2021

Collaborated with clients to troubleshoot software issues and query databases for diagnostics.
Documented and resolved over 500 technical support tickets with a 95% satisfaction rate.
Education
Master of Science in Artificial Intelligence
University of California, Berkeley | 2019

Bachelor of Technology in Computer Science
Indian Institute of Technology, Madras | 2017""",
    "job_description": """Die XXXLutz-Unternehmensgruppe betreibt über 370 Einrichtungshäuser in 14 europäischen Ländern und beschäftigt mehr als 27.100 Mitarbeiter. Mit einem Jahresumsatz von 6 Milliarden Euro gehört XXXLutz zu den größten Möbelhändlern der Welt. Zudem werden bereits 24 Onlineshops in den Vertriebsschienen XXXLutz, Möbelix und Mömax betrieben. Die Zentrale des 1945 gegründeten Familienunternehmens befindet sich in Wels.


Wir als XXXLdigital Team arbeiten gemeinsam an der Zukunft des Möbelhandels. Dabei arbeiten wir nicht nur in Teams, sondern leben sie auch – den Zusammenhalt, das Gefühl, das Lernen. Wir bringen unsere Ideen ein und gestalten proaktiv die Zukunft – und das auf allen digitalen Touchpoints. Wir sind begeistert und ständig auf der Suche nach neuen Lösungswegen. Dafür suchen wir Gleichgesinnte, die diesen Weg mit uns gehen wollen und bereit sind sich neuen Herausforderungen zu stellen.

Aufgaben:
Aufbau und Entwicklung der serviceorientierten XXXLutz AI Plattform und des dazu gehörigen CI/CD Prozesses für unsere Services
Entwicklung, Implementierung und Wartung der skalierbaren Infrastruktur für die Integration, Speicherung, Verarbeitung und Analyse von Daten auf Basis von Kubernetes und Kubeflow
Mitwirkung bei der Auswahlentscheidung der einzusetzenden Toolchain sowie deren Weiterentwicklung und Automatisierung mittels Scripting
Setup und Wartung von containerisierten Apps (Docker) und eingesetzten AI Tools sowie Implementieren von Security Konzepten für unsere AI Plattform
Mitarbeit an AI Projekten""",
    "job_title": "Data Scientist",
    "company_name": "Tech Corp",
    "history": [{"question": "What are your strengths?", "answer": "Problem-solving and teamwork"}]
})
agent_state["history"].append({"question": "What is your experience with Python?", "answer": "5 years"})
print(result)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


input_variables=['company_name', 'history', 'job_description', 'job_title', 'resume'] metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'interviewer_personal_question', 'lc_hub_commit_hash': 'eb77f5e41c8a773b0b6235e77ae9b7957e7d56628a04c42fe5fb1d43f56a1e1e'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['company_name', 'history', 'job_description', 'job_title', 'resume'], template='You are a world-class interviewer conducting an interview for the position of {job_title} at {company_name}. You have access to the candidate\'s {resume}, the {job_description}, and the {history} of previous questions and answers.\nYour task is to generate ONE insightful personal question for the interview. Focus on the candidate\'s background, experience, and resume details.\nConsider the following when generating your question:\n- Tailor the question to the specific job role and company.\n- Ensure the question aligns with the job requirements and assesses relevant skills.\n- Adju

InvalidUpdateError: Expected dict, got AgentState(resume='Name: Rahul Krishnamoorthy\nEmail: rahul.krish@example.com\nPhone: +1-234-567-8901\nLinkedIn: linkedin.com/in/rahulkrish\nGitHub: github.com/rahulkrish\n\nProfessional Summary\nDynamic AI Engineer and Full-Stack Developer with 5+ years of experience designing, developing, and deploying innovative software solutions. Proficient in backend development, API integration, and building data pipelines. Skilled in integrating GPT models and deploying CI/CD pipelines using AWS services.\n\nTechnical Skills\nProgramming Languages: Python, JavaScript, Java\nFrameworks & Tools: FastAPI, Flask, Vue.js, LangChain\nCloud Technologies: AWS (EKS, S3, SES, MSK)\nDatabases: MongoDB, PostgreSQL\nOther Skills: Kafka, CI/CD pipelines, Generative AI Integration\nExperience\nAI Engineer | Oros Tech LLC (Remote)\nJan 2022 – Present\n\nDeveloped over 50+ RESTful APIs using FastAPI and Flask.\nBuilt streaming pipelines using Kafka for real-time data proce